In [1]:
import torch
import torchvision
from typing import List
import numpy as np
import matplotlib.pyplot as plt
from time import time
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
from torch import nn, optim
from torch.utils.data import Dataset
import time
from tqdm import tqdm



# Es el nodo
class SplitNNServicer:
    def __init__(self, dataset_train, dataset_test):
        self.dataset = torch.utils.data.DataLoader(dataset_train, 
                                                   batch_size=64, 
                                                   shuffle=False)
        self.dataset_test = dataset_test
        self.all_servicers: List[SplitNNServicer] = []
        self.database = {}
    
    def FirstForward(self, images) -> torch.Tensor:
        out = self.database["model"](images)
        return out
        #return self.all_servicers[1].SecondForward(out)
        # es legal hacer self.all_servicers[1].Forward(...)
    
    def SecondForward(self, in_: torch.Tensor) -> torch.Tensor:
        return self.database["model"](in_)
    
    def SubirModelo(self, modelo, optimizador):
        self.database["model"] = modelo
        self.database["optimizer"] = optimizador

    def MetodoRandom(self, tensor_input: torch.Tensor) -> torch.Tensor:
        return tensor_input + self.global_dict["hola"]

In [4]:
import tempfile

# Create a temporary directory
temp_dir = tempfile.mkdtemp()



In [5]:
full_dataset = torchvision.datasets.MNIST(temp_dir, train=True, download=True,
    transform=torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(
            (0.1307,), (0.3081,))
        ]
    )
)

test_dataset = torchvision.datasets.MNIST(temp_dir, train=False, download=True,
    transform=torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(
            (0.1307,), (0.3081,))
        ]
    )
)

class AsdfDataset(Dataset):
    def __init__(self, dataset, index):
        self.dataset = dataset
        self.index = index

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        true_sample = self.dataset[idx]
        return true_sample[self.index]


features_dataset = AsdfDataset(full_dataset, 0)
labels_dataset = AsdfDataset(full_dataset, 1)

features_dataset_test = AsdfDataset(test_dataset, 0)
labels_dataset_test   = AsdfDataset(test_dataset, 1)

alice = SplitNNServicer(features_dataset,features_dataset_test)
bob = SplitNNServicer(labels_dataset, labels_dataset_test)

alice.all_servicers = [alice, bob]
bob.all_servicers = [alice, bob]

# A partir de ahora, solo podemos acceder a alice.MetodoEnMayusculas y bob.LoMismo

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:03<00:00, 2793306.72it/s]


Extracting /tmp/tmpf1r9p53y/MNIST/raw/train-images-idx3-ubyte.gz to /tmp/tmpf1r9p53y/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 316366.27it/s]


Extracting /tmp/tmpf1r9p53y/MNIST/raw/train-labels-idx1-ubyte.gz to /tmp/tmpf1r9p53y/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 2745232.54it/s]


Extracting /tmp/tmpf1r9p53y/MNIST/raw/t10k-images-idx3-ubyte.gz to /tmp/tmpf1r9p53y/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 2016142.32it/s]

Extracting /tmp/tmpf1r9p53y/MNIST/raw/t10k-labels-idx1-ubyte.gz to /tmp/tmpf1r9p53y/MNIST/raw



In [14]:
import csv

### client.py
class SplitNN_v1:
    def __init__(self, 
                    node_features: SplitNNServicer, model_features, optimizer_features,
                    node_labels: SplitNNServicer, model_labels, optimizer_labels,
    ):
        self.node_features = node_features
        self.model_features = model_features
        self.optimizer_features = optimizer_features
        
        self.node_features.SubirModelo(self.model_features, self.optimizer_features)
        
        
        self.node_labels = node_labels
        self.model_labels = model_labels
        self.optimizer_labels = optimizer_labels
        
        self.node_labels.SubirModelo(self.model_labels, self.optimizer_labels)


    def train(self, n_epochs):
        loss_per_epoch = []
        accuracy_per_epoch = []

        test_accuracy_per_epoch = []
        
        for i in tqdm(range(1,n_epochs+1), desc = "Epochs"):
            running_loss = 0
            correct = 0
            total = 0
            for images, labels in zip(self.node_features.dataset, self.node_labels.dataset):
                images = images.view(images.shape[0],-1)
                
                # 1) Zero our grads
                self.node_features.database['optimizer'].zero_grad()
                self.node_labels.database['optimizer'].zero_grad()
            
                # 2) Make a prediction
                a = []
                remote_a = []
                a.append(self.node_features.FirstForward(images))
                remote_a.append(a[-1].detach().requires_grad_())
                a.append(self.node_labels.SecondForward(remote_a[-1]))
                remote_a.append(a[-1].detach().requires_grad_())
                
                
                # 3) Figure out how much we missed by
                criterion = nn.NLLLoss()
                loss = criterion(a[-1], labels)
                
                #4) Backprop the loss on the end sub_model
                loss.backward()
                
                #5) Feed Gradients backward through the network
                grad_a = remote_a[0].grad
                a[0].backward(grad_a)
                
                #6) Change the weights
                self.node_features.database['optimizer'].step()
                self.node_labels.database['optimizer'].step()
                
                running_loss += loss.item()

                # Calculate accuracy
                _, predicted = torch.max(a[-1].data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                
            
            loss_per_epoch.append(running_loss/len(self.node_features.dataset))
            # print(f"Epoch {i} - Training loss: {running_loss/len(self.node_features.dataset)}")
            accuracy_per_epoch.append(100*correct/total)

            current_test_accuracy = self.test()
            test_accuracy_per_epoch.append(current_test_accuracy)


        
        # Save results to CSV
        with open("Train_metrics.csv", mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["Epoch", "Train Loss", "Train Accuracy"])
            for epoch in range(n_epochs):
                writer.writerow([epoch + 1, loss_per_epoch[epoch], accuracy_per_epoch[epoch]])
        
        with open("Test_metrics.csv", mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["Epoch", "Test Accuracy"])
            for epoch in range(n_epochs):
                writer.writerow([epoch + 1, test_accuracy_per_epoch[epoch]])

                
                
    
    def test(self):
        total = 0
        correct = 0
        for images, labels in zip(self.node_features.dataset_test, self.node_labels.dataset_test):
            images = images.view(images.shape[0], -1)
            a = []
            remote_a = []
            a.append(self.node_features.FirstForward(images))
            remote_a.append(a[-1].detach().requires_grad_())
            a.append(self.node_labels.SecondForward(remote_a[-1]))
            logits = a[-1]


            _, predicted = torch.max(logits.data, 1)
            total += 1
            correct += (predicted == labels).sum().item()
        
        test_accuracy = 100*correct/total
        # print(f'Test Accuracy: {test_accuracy:.2f}%')
        return  test_accuracy




In [15]:
### datascientist.py

import torch.nn as nn

torch.manual_seed(0)
input_size = 784
output_size = 10
hidden_sizes = [128, 640]

modelo_alice = nn.Sequential(
    nn.Linear(input_size, hidden_sizes[0]),
    nn.ReLU(),
    nn.Linear(hidden_sizes[0], hidden_sizes[1]),
    nn.ReLU(),
)

modelo_bob = nn.Sequential(
    nn.Linear(hidden_sizes[1], output_size),
    nn.LogSoftmax(dim=1)
)

optimizador_alice = optim.SGD(modelo_alice.parameters(), lr=0.03)
optimizador_bob   = optim.SGD(modelo_alice.parameters(), lr=0.03)


modelo = SplitNN_v1(
    alice, modelo_alice, optimizador_alice,
    bob, modelo_bob, optimizador_bob
)

modelo.test() # -> float del loss en el dataset entero
modelo.train(n_epochs=10) # -> None
modelo.test() # -> float del loss en el dataset entero

Epochs: 100%|██████████| 10/10 [04:42<00:00, 28.25s/it]


97.39